#### Etape 1.1 : Exploration initiale
- Charger les donnees de consommation avec PySpark
- Analyser le schema infere et identifier les problemes de typage
- Calculer les statistiques descriptives par type d'energie
- Identifier les batiments avec le plus de mesures
- Produire un rapport d'audit de qualite des donnees

**Livrables** :
- Notebook `01_exploration_spark.ipynb`
- Rapport d'audit (format markdown ou DataFrame)

In [8]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.master("local").appName("01_exploration").getOrCreate()

# Charger les donnees de consommation avec PySpark
df = spark.read \
          .option("header", "true") \
          .option("inferSchema", "true") \
          .option("sep", ",") \
          .csv("../data_ecf/consommations_raw.csv")

# Analyser le schema
df.printSchema()
df.show()

# Calculer les statistiques descriptives par type d'energie
df.groupBy("type_energie").agg(
    F.count("*").alias("count"),
    F.round(F.sum("consommation"),2).alias("total_consommation_kWh"),
    F.round(F.avg("consommation"),2).alias("avg_consommation_kWh")
).show()

# Identifier les batiments avec le plus de mesures
df.groupBy("batiment_id").count().orderBy(F.desc("count")).show()

# Produire un rapport d'audit de qualite des donnees
print("Nombre de lignes total : ", df.count())
print("Nombre de lignes avec valeurs manquantes : ", df.filter(F.col("consommation").isNull()).count())
print("Nombre de lignes avec valeurs nuls : ", df.filter(F.col("consommation") == 0).count())

root
 |-- batiment_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- type_energie: string (nullable = true)
 |-- consommation: string (nullable = true)
 |-- unite: string (nullable = true)

+-----------+-------------------+------------+------------+-----+
|batiment_id|          timestamp|type_energie|consommation|unite|
+-----------+-------------------+------------+------------+-----+
|    BAT0141|2023-12-21 13:00:00|         gaz|      342.34|  kWh|
|    BAT0080|   08/08/2023 13:00|         gaz|     1256.73|  kWh|
|    BAT0122|06/13/2024 11:00:00|         eau|      133.57|   m3|
|    BAT0033|2023-06-25 00:00:00|         eau|        0.23|   m3|
|    BAT0064|11/29/2024 04:00:00|         gaz|       12.26|  kWh|
|    BAT0052|09/15/2024 18:00:00|         gaz|      701.01|  kWh|
|    BAT0097|2024-01-25 12:00:00|         gaz|      444.87|  kWh|
|    BAT0073|   27/03/2023 13:00| electricite|      123.97|  kWh|
|    BAT0084|03/12/2024 17:00:00|         eau|       17.88|